<h2>Required Libraties</h2>

In [ ]:
!pip3 install sentence_transformers

In [26]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import regex as re
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import numpy as np
import torch

<h2>Data Source: Furniture Sales</h2>

Dataset containes 9994 entries and 12 features. For the Association Analysis the Category, Sub- Category and Product Name features will be look at and all three features will be grouped by the Order ID

In [8]:
# Data Source: 
# References: https://www-users.cse.umn.edu/~kumar001/dmbook/ch6.pdf, https://medium.com/edureka/apriori-algorithm-d7cc648d4f1e, https://medium.com/edureka/apriori-algorithm-d7cc648d4f1e, https://medium.com/@mervetorkan/association-rules-with-python-9158974e761a
df = pd.read_csv("./Superstore KPIs.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      9994 non-null   object 
 1   Order Date    9994 non-null   object 
 2   Region        9994 non-null   object 
 3   Segment       9994 non-null   object 
 4   Category      9994 non-null   object 
 5   Sub-Category  9994 non-null   object 
 6   Product Name  9994 non-null   object 
 7   Ship Date     9994 non-null   object 
 8   Ship Mode     9994 non-null   object 
 9   Profit        9994 non-null   float64
 10  Quantity      9994 non-null   int64  
 11  Sales         9994 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 937.1+ KB


<h2>Association Analysis: Preprocessing</h2>

* Need to remove odd characters and numbers from Product Name and trim whitespace. 

* Group all transactions by Order ID

* Create a separate dataframe with the Order ID and Product Name Instances as columns. uncomment line # print(basket_sets) if you want to see an example of the new dataset



In [ ]:
#Preprocessing 1: Clean up Product Name. Remove Numbers and odd characters
df['Product Name'] = df['Product Name'].str.replace('[^a-zA-Z ]', '')
df['Product Name'].str.strip()

#Preprocessing 2: Group by Order ID
def groupdataset(dataset,groupbysecondval):
  associated_df = (dataset.groupby(['Order ID', groupbysecondval])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Order ID'))
  return associated_df 

#Preprocessing 3: Convert all values that is not 0 to 1
def encode_units(x):
    return 1 if x >= 1 else 0

<h2>Association Analysis: Apiori Algorithm</h2>

In [251]:
#Apiori Algorithm
def apiori(dataset, minsupport):
  basket_sets = dataset.applymap(encode_units)
  frequent_itemsets = apriori(basket_sets, min_support=minsupport, use_colnames=True)
  rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
  # print(basket_sets)
  return rules

<h2>Association Analysis: Result 1 - Category</h2>

Confidence % are too low. Didn't learn much from it. The categories are too general too. Need to look at Sub Categories for more information.

In [252]:
associated_df = groupdataset(df,'Category')
rules = apiori(associated_df, 0.01)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Technology, Office Supplies)",(Furniture),0.182472,0.352166,0.064684,0.354486,1.006587,0.000423,1.003594
1,"(Office Supplies, Furniture)",(Technology),0.193452,0.308245,0.064684,0.334365,1.084738,0.005053,1.039241
2,(Technology),"(Office Supplies, Furniture)",0.308245,0.193452,0.064684,0.209845,1.084738,0.005053,1.020746
3,(Furniture),"(Technology, Office Supplies)",0.352166,0.182472,0.064684,0.183673,1.006587,0.000423,1.001472


<h2>Association Analysis: Result 2 - SubCategory</h2>

Need a further breakdown of the following categories: Appliances, Furnishings, Storage as they are still to general. 

In [12]:
associated_df = groupdataset(df,'Sub-Category')
rules = apiori(associated_df, 0.02)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Binders),(Appliances),0.262727,0.090038,0.025953,0.098784,1.097140,0.002298,1.009705
1,(Appliances),(Binders),0.090038,0.262727,0.025953,0.288248,1.097140,0.002298,1.035857
2,(Paper),(Appliances),0.237772,0.090038,0.021761,0.091520,1.016458,0.000352,1.001631
3,(Appliances),(Paper),0.090038,0.237772,0.021761,0.241685,1.016458,0.000352,1.005160
4,(Art),(Phones),0.145937,0.162507,0.024755,0.169631,1.043833,0.001040,1.008578
5,(Phones),(Art),0.162507,0.145937,0.024755,0.152334,1.043833,0.001040,1.007546
6,(Furnishings),(Phones),0.175085,0.162507,0.029547,0.168757,1.038458,0.001094,1.007518
7,(Phones),(Furnishings),0.162507,0.175085,0.029547,0.181818,1.038458,0.001094,1.008230
8,(Furnishings),(Storage),0.175085,0.155121,0.027550,0.157355,1.014401,0.000391,1.002651
9,(Storage),(Furnishings),0.155121,0.175085,0.027550,0.177606,1.014401,0.000391,1.003066


<h2>Association Analysis: Result 3 - Product Name</h2>

In [ ]:
#Checking Product Name - Confidence is too low. Need to generalize some of the descriptions of the Product Name
df2 = df
df2["ProductName_SubCategory"] = df[['Sub-Category','Product Name']].agg(' '.join, axis=1)
associated_df = groupdataset(df2,'ProductName_SubCategory')
rules = apiori(associated_df, 0.001) #Played with the min_support val and highest threshhold is 0.1%
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Paper Xerox ),(Accessories Kingston Digital DataTraveler GB ...,0.153124,0.004991,0.001198,0.007823,1.567353,0.000434,1.002854
1,(Accessories Kingston Digital DataTraveler GB ...,(Paper Xerox ),0.004991,0.153124,0.001198,0.240000,1.567353,0.000434,1.114310
2,(Art Newell ),(Paper Xerox ),0.050908,0.153124,0.007986,0.156863,1.024414,0.000190,1.004434
3,(Paper Xerox ),(Art Newell ),0.153124,0.050908,0.007986,0.052151,1.024414,0.000190,1.001311
4,(Envelopes Staple envelope),(Paper Xerox ),0.009583,0.153124,0.001597,0.166667,1.088440,0.000130,1.016251
5,(Paper Xerox ),(Envelopes Staple envelope),0.153124,0.009583,0.001597,0.010430,1.088440,0.000130,1.000856
6,(Fasteners Staples),(Paper Xerox ),0.009183,0.153124,0.001797,0.195652,1.277734,0.000391,1.052872
7,(Paper Xerox ),(Fasteners Staples),0.153124,0.009183,0.001797,0.011734,1.277734,0.000391,1.002581
8,(Paper Xerox ),"(Storage Hot File -Pocket, Floor Stand)",0.153124,0.002595,0.001397,0.009126,3.516498,0.001000,1.006591
9,"(Storage Hot File -Pocket, Floor Stand)",(Paper Xerox ),0.002595,0.153124,0.001397,0.538462,3.516498,0.001000,1.834897


<h2>Breaking down the Description of product names for the following sub categories</h2>


1.   Appliances
2.   Furnishing
3.   Storage
[link text](https://)

**Steps on how to further breakdown the subcategories**

1. Datasets where Product Name is appliable, furnishing or Storage 
need to be split into new datasets. 
2. Since we are working with Unsupervised Learning a model needs to be run to select particular words that will be be our classes/labels. The ML method Count Vector will be used for this. 
3. Once the labels have been set, a semantic similarity search model needs to match the new classes to the Product Names. The pretrained S-BERT model will be run. 
4. Group all the datasets again and run the Apiori model


**1) Split Datasets**

In [225]:
#Need to deploy NLP Model to generalize product name
#Reference: https://towardsdatascience.com/text-summarization-with-nlp-textrank-vs-seq2seq-vs-bart-474943efeb09
appliances_df = df[(df['Sub-Category'] == "Appliances")]
furnishing_df = df[(df['Sub-Category'] == "Furnishings")]
storage_df = df[(df['Sub-Category'] == "Storage")]

**2) Count Vector**

In [14]:
def extractclasses(dataset):
  productname1 =[]
  productname2 = []
  groupedproductname = []

  #Group sentences
  sentences = dataset['Product Name']
  for i,j in enumerate(sentences):
    x = j.split()
    for y in x:
      productname1.append(y)
      
  [productname2.append(j) for i,j in enumerate(sentences)]
  groupedproductname = productname1 + productname2

  #Run CountVector
  cv=CountVectorizer()
  vect=cv.fit_transform(groupedproductname)
  km=KMeans(n_clusters=100)
  pred = km.fit_predict(vect)

  classnames = []
  for i,j in enumerate(pred):
    classnames.append(groupedproductname[j])

  classnames = set(classnames)
  print(*list(classnames), sep = "\n")

**3) Select the Classes/Labels**

In [15]:
extractclasses(appliances_df)

HEPA
HOMEOFFICE
Outlet
Air
HomeOffice
Fellowes
Amp
Strip
Hard
FaxPhone
Commercial
II
Replacement
Enviracaire
Phone
Large
Very
Sanyo
Office
Cube
Megaboss
Compact
Bravo
per
Acco
Belkin
MidSize
SixOutlet
Series
Belts
Protectors
Protection
Body
Bags
Filter
for
Foot
Surge
Masterpiece
x
Power
Refrigerators
Cleaner
Cord
Room
Vibra
Holmes
Sanitaire
Portable
Line
Cubic
Length
Disposable
SurgeMaster
Honeywell
Center
Upright
Control
with
Kensington
Protector
Pack
Wihtout
Basic
Eureka
MasterPiece
FCVTEL


In [16]:
extractclasses(furnishing_df)

Carpets
High
Chairmats
Beveled
DuraMat
Soft
Clock
Chair
Traditional
Lamp
Expressions
Plus
Wall
Cleatmat
Cubicle
Accessory
Arm
Pencil
Artistic
Holder
Medium
Mat
Chrome
ClampOn
LongerLife
Oak
Economy
Desk
for
Thomas
x
Standard
Pile
White
Diameter
Deflecto
Architects
Swing
Cherry
Howard
InstaPlaque
Lip
Tenex
Brushed
and
Studded
Bulbs
Mats
Eldon
Electrix
Seth
Black
Carpeting
Luxo
Round
Magnifier
Carpet
Lighweight
Wood
Accessories
Miller
Low
Plastic


In [17]:
extractclasses(storage_df)

shelf
StorDrawer
HomeOffice
Panel
Fellowes
N
Fold
II
Vertical
Shelving
Smoke
H
Safco
System
Cart
Super
Chrome
Companion
h
Manager
Front
Pocket
Sortfiler
Drawer
LetterLegal
Personal
Drawers
Wire
storage
Advantus
Multipurpose
Carts
Frame
Base
for
stackable
x
W
D
Metal
Organizer
Tenex
Bin
Plastics
Gould
Mobile
Shelf
Eldon
Portable
File
TravLFile
SturDStor
Design
w
Black
Industrial
Roll
SimpliFile
platinum
Scoop
Granite
with
d
Project
HeavyDuty
Shuttle


**3) S-BERT: Semantic Textual Similarity**

In [226]:
def semanticSimilarityFunction(dataset, compareList):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  productname2 = []
  tempdictionary = {}
  sentences = dataset['Product Name']
  [productname2.append(j) for i,j in enumerate(sentences)]

  #Compute embedding for both lists
  embeddings1 = model.encode(productname2, convert_to_tensor=True)
  embeddings2 = model.encode(compareList, convert_to_tensor=True)

  #Compute cosine-similarities
  cosine_scores = util.cos_sim(embeddings1, embeddings2)

  k = torch.topk(cosine_scores, 1, dim=1, largest=True, sorted=True)

  for j in compareList:
    tempdictionary[j] = []
  for i in range(len(productname2)):
          print("Key Word:{} \tSuggested Class(label):{} \tScore:{}".format(productname2[i],compareList[k.indices[[i]]], k.values[[i]]))
          tempdictionary[compareList[k.indices[i]]].append([productname2[i]])
  return tempdictionary

**(New Labels/Classes)**

In [163]:
appliancesClasses = [
'Filter',
'Dirt/Dust Bags',
'Outlet/Protectors',
'Phone',
'Cord',
'Belts',
'Cleaner',
'Air Cleaner/Air Purifiers/Humidifier',
'Refrigerators',
'FaxPhone',
'Vacuum',
'Staple holder'
]
furnishingClasses = [
'Chair/Clear Mat',
'Accessories',
'Pencil',
'Clock',
'Lamp',
'ClampOn',
'Chair',
'Wall',
'Magnifier',
'Frame',
'Door handle',
'Light Bulbs'
]
storageClasses = [
'Shelf',
'Organiser',
'Frame',
'Cart',
'Storage',
'Letter',
'Panel',
'File',
'Locker',
'Roll',
'Drawer',
'Folder',
'StorDrawer'
]

In [ ]:
storagedictionary = semanticSimilarityFunction(storage_df,storageClasses)

In [ ]:
appliancedictionary = semanticSimilarityFunction(appliances_df,appliancesClasses)

In [ ]:
furnishingdictionary = semanticSimilarityFunction(furnishing_df,furnishingClasses)

**4) Results**

In [227]:
def replaceProductName(dataset, replaceDataset):
  for x in replaceDataset:
    for values in replaceDataset[x]:
      for index,y in enumerate(dataset['Product Name']):
        if y == values[0]:
          dataset['Product Name'].loc[dataset['Product Name'] == y] = x

In [ ]:
replaceProductName(furnishing_df,furnishingdictionary)
replaceProductName(appliances_df,appliancedictionary)
replaceProductName(storage_df,storagedictionary)

In [241]:
others_df =  df[((df['Sub-Category'] != "Appliances") & (df['Sub-Category'] != "Furnishings") & (df['Sub-Category'] != "Storage"))]

frames = [others_df, furnishing_df, appliances_df, storage_df]
all_df = pd.concat(frames)

In [246]:
#Run the apiori model again
all_df["ProductName_SubCategory2"] = all_df[['Sub-Category','Product Name']].agg(' '.join, axis=1)
associated_df = groupdataset(all_df,'ProductName_SubCategory2')
rules = apiori(associated_df, 0.005)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Appliances Outlet/Protectors),(Paper Xerox ),0.042523,0.153124,0.006788,0.159624,1.042449,0.000276,1.007735
1,(Paper Xerox ),(Appliances Outlet/Protectors),0.153124,0.042523,0.006788,0.044329,1.042449,0.000276,1.001889
2,(Paper Xerox ),(Art Newell ),0.153124,0.050908,0.007986,0.052151,1.024414,0.000190,1.001311
3,(Art Newell ),(Paper Xerox ),0.050908,0.153124,0.007986,0.156863,1.024414,0.000190,1.004434
4,(Furnishings Frame),(Paper Xerox ),0.034737,0.153124,0.006588,0.189655,1.238569,0.001269,1.045081
5,(Paper Xerox ),(Furnishings Frame),0.153124,0.034737,0.006588,0.043025,1.238569,0.001269,1.008660
6,(Storage File),(Paper Xerox ),0.035536,0.153124,0.005590,0.157303,1.027292,0.000149,1.004959
7,(Paper Xerox ),(Storage File),0.153124,0.035536,0.005590,0.036506,1.027292,0.000149,1.001007
8,(Storage Shelf),(Paper Xerox ),0.030745,0.153124,0.006189,0.201299,1.314609,0.001481,1.060316
9,(Paper Xerox ),(Storage Shelf),0.153124,0.030745,0.006189,0.040417,1.314609,0.001481,1.010080
